# Computational Theory 

## Problem 1 — Binary Words and Operations

Implement the SHA-256 bitwise functions: `Parity`, `Ch`, `Maj`, `Sigma0`, `Sigma1`, `sigma0`, `sigma1`.

Below we provide:

- Clear docstrings for each function.
- Explanations of the logical behaviour (algorithmic steps)
- Tests comparing small examples and showing values in hex.


In [ ]:

import numpy as np

# Helper for rotating 32-bit integers
def _rotr(x: np.uint32, n: int) -> np.uint32:
    """Rotate-right (32-bit) helper."""
    return np.uint32((x >> n) | (x << (32 - n)))

# Bitwise operations used in SHA-like algorithms
def Parity(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """Parity(x,y,z) = x XOR y XOR z."""
    return np.uint32(x ^ y ^ z)

def Ch(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """Ch(x,y,z) = (x AND y) XOR ((NOT x) AND z)."""
    return np.uint32((x & y) ^ (~x & z))

def Maj(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """Maj(x,y,z) = (x AND y) XOR (x AND z) XOR (y AND z)."""
    return np.uint32((x & y) ^ (x & z) ^ (y & z))


In [ ]:
# Uppercase Sigma functions (used in main compression)
def Sigma0(x: np.uint32) -> np.uint32:
    """Σ0(x) = ROTR^2(x) XOR ROTR^13(x) XOR ROTR^22(x)."""
    return np.uint32(_rotr(x, 2) ^ _rotr(x, 13) ^ _rotr(x, 22))

def Sigma1(x: np.uint32) -> np.uint32:
    """Σ1(x) = ROTR^6(x) XOR ROTR^11(x) XOR ROTR^25(x)."""
    return np.uint32(_rotr(x, 6) ^ _rotr(x, 11) ^ _rotr(x, 25))

# Lowercase sigma functions (used for message schedule)
def sigma0(x: np.uint32) -> np.uint32:
    """σ0(x) = ROTR^7(x) XOR ROTR^18(x) XOR SHR^3(x)."""
    return np.uint32(_rotr(x, 7) ^ _rotr(x, 18) ^ (x >> 3))

def sigma1(x: np.uint32) -> np.uint32:
    """σ1(x) = ROTR^17(x) XOR ROTR^19(x) XOR SHR^10(x)."""
    return np.uint32(_rotr(x, 17) ^ _rotr(x, 19) ^ (x >> 10))
